### Import TensorFlow2 and other libraries

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

구글 드라이브를 마운트하고, 패키지를 설치할 경로를 설정합니다.

/content/mnt/My Drive/Colab Notebooks/packages 폴더를 먼저 만들고, content/notebooks/packages라는 경로에 매핑합니다.



In [1]:
import os
import sys
from google.colab import drive
drive.mount('/content/mnt')
# drive.mount("/content/mnt", force_remount=True)
nb_path = '/content/notebooks_packages'
sys.path.insert(0, nb_path)
os.symlink('/content/mnt/My Drive/Colab Notebooks/packages', nb_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/mnt


텐서플로우 2.0 gpu버전을 nb-path에 설치합니다. 최초 한번만 실행하고 이후부터는 이 라인은 스킵해서 실행합니다.

설치한 뒤에 런타임을 한번 재시작해줍니다.

로컬 런타임이었다면

!pip install Pillow 

!pip install matplotlib 

!pip install imageio 

!pip install opencv-python


도 함께 실행해야했겠지만, Colab에 이미 설치되어있으므로 스킵합니다.

In [0]:
!pip install --target=$nb_path --upgrade tensorflow-gpu==2.0.0-beta1


import를 실행하면 시간이 좀 걸린뒤 완료됩니다. (10~20분)

In [0]:
import tensorflow as tf

텐서플로우 버전과 사용하고 있는 GPU 이름을 확인하여 정상적으로 설치되고, 적합한 세션을 사용하고 있는지 확인합니다.

2.0.0-beta1
GPU name : ~~~

이 출력되면 정상 세팅된 상태입니다.

In [2]:
print(tf.__version__)

from tensorflow.python.client import device_lib

"GPU" + device_lib.list_local_devices()[-1].physical_device_desc.split(",")[1]

NameError: ignored

나머지 필요한 패키지 세팅

In [0]:
import time
import datetime
import random
import os
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import imageio
from PIL import Image
import cv2
import PIL

from IPython import display

%matplotlib inline

In [0]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

### 데이터셋을 로드하고 학습가능한 형태로 정리합니다.

OUTPUT_DIR를 설정할 때, 잘못설정하면 수정하기 아주 귀찮은 디렉토리에 저장되기 때문에 항상 확인하고 학습을 진행하도록 합니다.

In [0]:
# Paths Setting
INPUT_DATA_DIR = '/content/mnt/My Drive/Colab Notebooks/dataset/simpson/'
OUTPUT_DIR = '/content/mnt/My Drive/Colab Notebooks/results/simpson/output_{date:%Y-%m-%d_%H%M%S}/'.format(date=datetime.datetime.now())
# OUTPUT_DIR = '/content/mnt/My Drive/Colab Notebooks/results/simpson/output_2019-12-20_024720/'

OUTPUT_LOSS_DIR = OUTPUT_DIR + 'loss/'
OUTPUT_IMG_DIR = OUTPUT_DIR + 'images/'
OUTPUT_WEIGHT_DIR = OUTPUT_DIR + 'weight/'

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

if not os.path.exists(OUTPUT_LOSS_DIR):
    os.makedirs(OUTPUT_LOSS_DIR)

if not os.path.exists(OUTPUT_IMG_DIR):
    os.makedirs(OUTPUT_IMG_DIR)
    
if not os.path.exists(OUTPUT_WEIGHT_DIR):
    os.makedirs(OUTPUT_WEIGHT_DIR)    

추론률을 떨어트리는 데이터셋을 제외하고 파일을 로드합니다

In [0]:
exclude_file_list = ["9746","9731","9717","9684","9637","9641","9642","9584","9541","9535",
                     "9250","9251","9252","9043","8593","8584","8052","8051","8008","7957",
                     "7958","7761","7762","9510","9307","4848","4791","4785","4465","2709",
                     "7724","7715","7309","7064","7011","6961","6962","6963","6960","6949",
                     "6662","6496","6409","6411","6406","6407","6170","6171","6172","5617",
                     "4363","4232","4086","4047","3894","3889","3493","3393","3362","2780",
                     "2710","2707","2708","2711","2712","2309","2056","1943","1760","1743",
                     "1702","1281","1272","772","736","737","691","684","314","242","191"]

for ex_image in exclude_file_list:
    if os.path.exists(f'{INPUT_DATA_DIR}{ex_image}.png'):
        print(f'{INPUT_DATA_DIR}{ex_image}.png')
        os.rename(f'{INPUT_DATA_DIR}{ex_image}.png', f'{INPUT_DATA_DIR}{ex_image}.png_bak')


학습에 사용될 하이퍼 파라미터들

가장 어려운 부분이 시작된다! 컨볼루션(합성곱) 네트워크 용어를 이해하고 넘어가보자

**합성곱(convolution)**

이렇게 부분부분을 곱해서 정리하는 것을 합성곱이라고 한다.
![합성곱 작동 원리](https://studio-velcro.com/img/Convolution_schematic.gif)

**채널**

RGB할 때 그런 채널. 이미지뿐만 아니라 여러가지 모든 레이어는 다 채널로 표현한다.

**필터 & 스트라이드**

필터는 이미지에서 feature map을 뽑아낼 틀을 이야기함. 보통 정사각형 모양을 가진다. 합성곱에 있는 gif참조.
![대체 텍스트](https://studio-velcro.com/img/conv_filter.png)
스트라이드(stride)는 필터를 몇칸씩 움직이며 feature map을 작성할 것인가를 결정하는 파라미터. 1이면 1칸씩 2면 2칸씩 ....

멀티채널의 경우 아래와같이 계산해서 이미지에 대한 최종적인 feature map을 만들어낸다.
![대체 텍스트](https://studio-velcro.com/img/conv2_filter_multichannel.jpg)

**패딩**

feature map을 만들고나면 결과물은 실제 이미지보다 사이즈가 작아지게 된다! 이미지 사이즈를 맞춰주기위해 feature map 주변에 0(보통 0임)으로 둘러 사이즈를 맞추게 된다. 이미지 외곽을 학습시키는 부가효과도 있다.
![대체 텍스트](https://studio-velcro.com/img/padding.png)

**pooling**

출력 데이터를 강조하거나 크기를 줄이는 데 사용한다. 그런게 있다 정도만 이해하면 될듯.
CNN에서는 주로 Max Pooling을 사용한다.
![대체 텍스트](https://studio-velcro.com/img/maxpooling.png)

용어는 여기까지 알아보고 더 자세한 내용은 좀 이따 제너레이터/디스크리미네이터 짤 때 추가로 알아보자

배치사이즈 64 : 한번에 몇장의 이미지를 학습할 것인가 -> 64장씩 학습할거다.


In [0]:
# Hyperparameters
IMAGE_SIZE = 128
NOISE_SIZE = 100
LR_D = 0.00004
LR_G = 0.0004
BATCH_SIZE = 64
EPOCHS = 300
BETA1 = 0.5
WEIGHT_INIT_STDDEV = 0.02
EPSILON = 0.00005
SAMPLES_TO_SHOW = 16    # 5 > 16
BUFFER_SIZE = 1000     # 10240 ?

이미지에 적용할 노이즈를 생성합니다.
16x100 형태의 노이즈 생성

16: 샘플 이미지 갯수

100: 노이즈 사이즈

In [0]:
display_noise = tf.random.uniform(shape=[SAMPLES_TO_SHOW, NOISE_SIZE], minval=-1, maxval=1, dtype=tf.dtypes.float32)
print(display_noise.shape)

w_init = tf.initializers.TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)
w_init

In [0]:
def load_data(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img)
    img = tf.image.resize(img, size=[IMAGE_SIZE, IMAGE_SIZE])[..., :3]
    img /= 127.5
    img -= 1
    return img

In [0]:
image_list = glob(INPUT_DATA_DIR + '*.png')
print('Found {} images'.format(len(image_list)))


In [0]:
image_dataset = tf.data.Dataset.from_tensor_slices(image_list)

In [0]:
image_dataset = image_dataset.shuffle(buffer_size=BUFFER_SIZE)

In [0]:
image_dataset = image_dataset.map(load_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
image_dataset = image_dataset.batch(BATCH_SIZE)

In [0]:
image_dataset = image_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

## Create the models



### The Generator (더 어려운 이야기의 시작)

레이어 사이즈를 결정할 때는 공식에 따라야한다. 그냥 파라미터를 어떻게 정하면 어떻게 출력된다를 정리한거니까 이해하려고하지 말자.

입력 데이터 -> 필터+스트라이드+패딩 -> 출력 데이터

*   입력 데이터 높이: H
*   입력 데이터 폭: W
*   필터 높이: FH
*   필터 폭: FW
*   Stride 크기: S
*   패딩 사이즈: P
![대체 텍스트](https://studio-velcro.com/img/conv_equation.png)

이 식의 결과는 모두 **자연수**여야한다. 컨볼루션 레이어 뒤에 풀링 레이어가 오면 Feature Map의 행과 열 크기는 Pooling 크기의 배수여야 한다. Pooling 사이즈가 (3, 3)이라면 위 식의 결과는 자연수이고 3의 배수여야 한다. 이 조건을 만족하도록 필터와 stride, pooling 사이즈를 조정해야한다. 왜냐? 안맞으면 데이터가 안들어간다!





In [0]:
def make_generator_model(generator_dim=100):
    
    noise = tf.keras.layers.Input(shape=(generator_dim,), name='generator_noise_input')
    
    # 1024x8x8 = 262,144
    x = tf.keras.layers.Dense(8*8*1024)(noise)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU()(x)
    # 8x8x1024
    x = tf.keras.layers.Reshape(target_shape=[8, 8, 1024])(x)
    print(x.shape)
    
    # 1. 16x16x1024 -> 32x32x512
    x = tf.keras.layers.Conv2DTranspose(filters=512,
                                        kernel_size=5,
                                        strides=2,
                                        padding='same',
                                        kernel_initializer=w_init,
                                        use_bias=True)(x)
    x = tf.keras.layers.BatchNormalization(epsilon=EPSILON)(x)
    x = tf.keras.layers.ReLU()(x)
    print(x.shape)
    
    # 2. 32x32x512 -> 64x64x256
    x = tf.keras.layers.Conv2DTranspose(filters=256,
                                        kernel_size=5,
                                        strides=2, 
                                        padding='same',
                                        kernel_initializer=w_init,
                                        use_bias=True)(x)
    x = tf.keras.layers.BatchNormalization(epsilon=EPSILON)(x)
    x = tf.keras.layers.ReLU()(x)
    print(x.shape)
    
    # 3. 64x64x256 -> 128x128x128
    x = tf.keras.layers.Conv2DTranspose(filters=128,
                                        kernel_size=5,
                                        strides=2, 
                                        padding='same',
                                        kernel_initializer=w_init,
                                        use_bias=True)(x)
    x = tf.keras.layers.BatchNormalization(epsilon=EPSILON)(x)
    x = tf.keras.layers.ReLU()(x)
    print(x.shape)
    
    # 4. 128x128x128 -> 256x256x64
    x = tf.keras.layers.Conv2DTranspose(filters=64,
                                        kernel_size=5,
                                        strides=2, 
                                        padding='same',
                                        kernel_initializer=w_init,
                                        use_bias=True)(x)
    x = tf.keras.layers.BatchNormalization(epsilon=EPSILON)(x)
    x = tf.keras.layers.ReLU()(x)
    print(x.shape)
    
    # 256x256x64 -> 512x512x32
    # 256x256x64 -> 512x512x3
    x = tf.keras.layers.Conv2DTranspose(filters=3,
                                        kernel_size=5,
                                        strides=1, 
                                        padding='same',
                                        kernel_initializer=w_init,
                                        use_bias=True)(x)
    print(x.shape)
    
    fake_output = tf.keras.layers.Activation('tanh', name='logits')(x)
    print(fake_output)
    
    return tf.keras.Model(inputs=[noise], outputs=[fake_output], name='Generator')


In [0]:
generator = make_generator_model(NOISE_SIZE)

generated_image = generator(display_noise, training=False)
print ("Input: " + str(generated_image.shape))

images = tf.clip_by_value((generated_image + 1) * 127.5, 0, 255).numpy()
for i in range(16):
    cv2.imwrite(f'{OUTPUT_IMG_DIR}generated_noise_{i}.png', cv2.cvtColor(images[i], cv2.COLOR_RGB2BGR))

### The Discriminator



In [0]:
def make_discriminator_model(latent_dim=100):
    
    image_input = tf.keras.layers.Input(shape=[IMAGE_SIZE, IMAGE_SIZE, 3], name='discriminator_image_input')
    print(image_input.shape)
    
    # 128*128*3 -> 64x64x64 
    x = tf.keras.layers.Conv2D(filters=64,
                               kernel_size=5,
                               strides=2, 
                               padding='same',
                               kernel_initializer=w_init,
                               use_bias=True)(image_input)
    x = tf.keras.layers.BatchNormalization(epsilon=EPSILON)(x)
    x = tf.keras.layers.LeakyReLU()(x)
    print(x.shape)
    
    # 64x64x64-> 32x32x128 
    x = tf.keras.layers.Conv2D(filters=128,
                               kernel_size=5,
                               strides=2, 
                               padding='same',
                               kernel_initializer=w_init,
                               use_bias=True)(x)
    x = tf.keras.layers.BatchNormalization(epsilon=EPSILON)(x)
    x = tf.keras.layers.LeakyReLU()(x)
    print(x.shape)

    # 32x32x128 -> 16x16x256  
    x = tf.keras.layers.Conv2D(filters=256,
                               kernel_size=5,
                               strides=2, 
                               padding='same',
                               kernel_initializer=w_init,
                               use_bias=True)(x)
    x = tf.keras.layers.BatchNormalization(epsilon=EPSILON)(x)
    x = tf.keras.layers.LeakyReLU()(x)
    print(x.shape)

    # 16x16x256 -> 16x16x512
    x = tf.keras.layers.Conv2D(filters=512,
                               kernel_size=5,
                               strides=1, 
                               padding='same',
                               kernel_initializer=w_init,
                               use_bias=True)(x)
    x = tf.keras.layers.BatchNormalization(epsilon=EPSILON)(x)
    x = tf.keras.layers.LeakyReLU()(x)
    print(x.shape)
    
    # 16x16x512 -> 8x8x1024
    x = tf.keras.layers.Conv2D(filters=1024,
                               kernel_size=5,
                               strides=2,
                               padding='same',
                               kernel_initializer=w_init,
                               use_bias=True)(x)
    x = tf.keras.layers.BatchNormalization(epsilon=EPSILON)(x)
    x = tf.keras.layers.LeakyReLU()(x)
    print(x.shape)
    
    x = tf.keras.layers.Reshape(target_shape=(-1, 8*8*1024))(x)
    logits = tf.keras.layers.Dense(units=1, activation=None)(x)
    print(logits)
    
    return tf.keras.Model(inputs=[image_input], outputs=[logits], name='Discriminator')


In [0]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision.shape)
#print (decision)

## Define the loss and optimizers




### Discriminator loss



In [0]:
def d_loss_real_function(d_logits_real):
    
    # Discriminator logic
    d_model_real = tf.sigmoid(d_logits_real)
    
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real,
                                                                         labels=tf.ones_like(d_model_real)*random.uniform(0.9, 1.0)))
    
    return d_model_real, d_loss_real


def d_loss_fake_function(d_logits_fake):
    
    # Discriminator logic
    d_model_fake = tf.sigmoid(d_logits_fake)
    
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                                         labels=tf.zeros_like(d_model_fake)))
    
    return d_model_fake, d_loss_fake

### Generator loss



In [0]:
def g_loss_function(d_logits_fake, d_model_fake):
    
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake,
                                                                    labels=tf.ones_like(d_model_fake)))
    return g_loss


### optimizers



In [0]:
d_optimizer = tf.keras.optimizers.Adam(learning_rate=LR_D, beta_1=BETA1)
g_optimizer = tf.keras.optimizers.Adam(learning_rate=LR_G, beta_1=BETA1)


### Save checkpoints




In [0]:
checkpoint_dir = OUTPUT_DIR + 'training_checkpoints'

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)    

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=g_optimizer,
                                 discriminator_optimizer=d_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Define the training loop



In [0]:
print(display_noise)


In [0]:
@tf.function
def training_step(images):
    
    noise = tf.random.uniform(shape=[BATCH_SIZE, NOISE_SIZE], minval=-1, maxval=1, dtype=tf.dtypes.float32)

    with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape, tf.GradientTape() as r_tape:

        # noise generator > discriminator
        g_model = generator(noise, training=True)
        
        d_logits_fake = discriminator(g_model, training=True)
        
        d_model_fake, d_loss_fake = d_loss_fake_function(d_logits_fake)
        
        
        # noise readl > discriminator 
        noisy_input_real = images + tf.random.normal(shape=tf.shape(images),
                                                     mean=0.0,
                                                     stddev=random.uniform(0.0, 0.1),
                                                     dtype=tf.dtypes.float32)
        
        d_logits_real = discriminator(noisy_input_real, training=True)
        
        d_model_real, d_loss_real = d_loss_real_function(d_logits_real)

        
        # discriminator loss
        d_loss = tf.reduce_mean(0.5 * (d_loss_real + d_loss_fake))
        
        # generator loss
        g_loss = g_loss_function(d_logits_fake, d_model_fake)

        discriminator_gradients_fake = d_tape.gradient(d_loss, discriminator.trainable_variables)
        generator_gradients = g_tape.gradient(g_loss, generator.trainable_variables)

        d_optimizer.apply_gradients(zip(discriminator_gradients_fake, discriminator.trainable_variables))
        g_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))

    return g_loss, d_loss

In [0]:
def train(epochs=10, save_every=3, steps=None):
    
    d_losses = []
    g_losses = []
    summary_step = 0
    
    for ep in range(epochs):
        
        start = time.time()
        
        running_loss = {'g_loss': [], 'd_loss': []}
        for step, images in enumerate(image_dataset):
            
            batch_g_loss, batch_d_loss = training_step(images)
            
            # loss write
            d_losses.append(batch_d_loss.numpy())
            g_losses.append(batch_g_loss.numpy())
            
            if step % 10 == 0:
                ctime = time.strftime("%Y/%m/%d/ %H:%M:%S")
                print(f'||epoch {ep}/{epochs} step {step+1}/{steps}|G_LOSS : {batch_g_loss:.3f}|D_LOSS : {batch_d_loss:.3f}|| {ctime}' )
            
            tf.summary.scalar("generator_batch_loss", batch_g_loss, step=summary_step + 1)
            tf.summary.scalar("discriminator_batch_loss_total", batch_d_loss, step=summary_step + 1)
            writer.flush()
            
            summary_step = summary_step + 1
        
        if ep % save_every == 0:
            print(f'||saving weights for epoch : {ep}||')
            generator.save_weights(f'{OUTPUT_WEIGHT_DIR}generator_weights_{ep}.h5')
            discriminator.save_weights(f'{OUTPUT_WEIGHT_DIR}discriminator_weights_{ep}.h5')
            checkpoint.save(file_prefix = checkpoint_prefix)
        
        display.clear_output(wait=True)
        print ('Time for epoch [{}] is [{}] sec'.format(ep, time.time()-start))

        # test noise not change for gif gen
        #display_noise = tf.random.uniform(shape=[SAMPLES_TO_SHOW, NOISE_SIZE], minval=-1, maxval=1, dtype=tf.dtypes.float32)
        save_generated_images(display_noise, epoch=ep)
        
        summarize_epoch(ep, time.time()-start, d_losses, g_losses)
            
    return d_losses, g_losses

**Generate and save images**




In [0]:
def save_generated_images(noise, epoch=None):
    
    images_result = generator(noise)
    
    # cv 
    images = tf.clip_by_value((images_result + 1) * 127.5, 0, 255).numpy()
    for i in range(16):
        cv2.imwrite(f'{OUTPUT_IMG_DIR}{i}_{epoch}.png', cv2.cvtColor(images[i], cv2.COLOR_RGB2BGR))
    
    # plt
    sample_images = [tf.cast((sample + 1) * 127.5, dtype=tf.uint8).numpy() for sample in images_result]
    fig = plt.figure(figsize=(6,6)) 

    for i in range(16):
        plt.subplot(4, 4, i+1)
        plt.imshow(sample_images[i])
        plt.axis('off')
    plt.savefig(OUTPUT_DIR + 'image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()
    plt.close()
    

In [0]:
def summarize_epoch(epoch, duration, d_losses, g_losses):
    
    minibatch_size = int(len(image_list)//BATCH_SIZE)
    
    print("Epoch {}/{}".format(epoch, EPOCHS),
          "\nDuration: {:.5f}".format(duration),
          "\nD Loss: {:.5f}".format(np.mean(d_losses[-minibatch_size:])),
          "\nG Loss: {:.5f}".format(np.mean(g_losses[-minibatch_size:])))
    
    fig, ax = plt.subplots()
    plt.plot(d_losses, label='Discriminator', alpha=0.6)
    plt.plot(g_losses, label='Generator', alpha=0.6)
    plt.title("Losses")
    plt.legend()
    plt.savefig(OUTPUT_LOSS_DIR + "losses_" + str(epoch) + ".png")
    plt.show()
    plt.close()


## Train the model



In [0]:
%%time
writer = tf.summary.create_file_writer(OUTPUT_DIR + 'logs')
with writer.as_default():
    steps_per_epoch = len(image_list)//BATCH_SIZE
    d_losses, g_losses = train(EPOCHS, 10, steps_per_epoch)

## Restore the latest checkpoint.

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Create a GIF


In [0]:
# Display a single image using the epoch number
def display_image(epoch_no):
    return PIL.Image.open('{}image_at_epoch_{:04d}.png'.format(OUTPUT_DIR, epoch_no))

In [0]:
display_image(0)

휴먼 소팅..ㅠ

In [0]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [0]:
anim_file = OUTPUT_DIR + 'dcgan_simsons_faces_masa.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
    
    filenames = glob(OUTPUT_DIR+'images/15_*.png')
    filenames = sorted(filenames)
    filenames.sort(key=natural_keys)
    
    last = -1
    for i,filename in enumerate(filenames):
        # frame = 2*(i**0.5)
        # if round(frame) > round(last):
        #     last = frame
        # else:
        #     continue
        
        # image = imageio.imread(filename)
        # writer.append_data(image)
        
        image = imageio.imread(filename)
        writer.append_data(image)



import IPython

if IPython.version_info > (6,2,0,''):
    display.Image(filename=anim_file)

In [0]:
anim_lfile = OUTPUT_DIR + 'dcgan_loss.gif'

with imageio.get_writer(anim_lfile, mode='I') as writer:
    
    filenames = glob(OUTPUT_LOSS_DIR+'losses_*.png')
    filenames = sorted(filenames)
    filenames.sort(key=natural_keys)
    
    # last = -1
    for i,filename in enumerate(filenames):
        # frame = 2*(i**0.5)
        # if round(frame) > round(last):
        #     last = frame
        # else:
        #     continue
        
        # image = imageio.imread(filename)
        # writer.append_data(image)
        
        image = imageio.imread(filename)
        writer.append_data(image)

import IPython

if IPython.version_info > (6,2,0,''):
    display.Image(filename=anim_lfile)

In [0]:
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download(anim_file)